In [11]:
import pandas as pd

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('../src/data/Bicycle_Thefts.csv')

print(df['PREMISES_TYPE'].value_counts())

def clean_bicycle_data(df):
    df_clean = df.copy()
    df_clean.loc[df_clean['PREMISES_TYPE'].isin(['Apartment', 'House']), 'PREMISES_TYPE'] = 'Residential'
    df_clean.loc[df_clean['PREMISES_TYPE'] == 'Transit', 'PREMISES_TYPE'] = 'Other'
    df_clean.loc[df_clean['PREMISES_TYPE'] == 'Educational', 'PREMISES_TYPE'] = 'Commercial'

    
    datetime_columns = ['OCC_DATE', 'REPORT_DATE']
    for col in datetime_columns:
        df_clean[col] = pd.to_datetime(df_clean[col])
    
    df_clean['BIKE_COST'] = pd.to_numeric(df_clean['BIKE_COST'], errors='coerce')
    df_clean['BIKE_COST'] = df_clean['BIKE_COST'].fillna(0)
    
    df_clean['BIKE_SPEED'] = pd.to_numeric(df_clean['BIKE_SPEED'], errors='coerce')
    df_clean['BIKE_SPEED'] = df_clean['BIKE_SPEED'].fillna(df_clean['BIKE_SPEED'].median())
    
    df_clean['BIKE_TYPE'] = df_clean['BIKE_TYPE'].fillna('UNKNOWN')
    
    df_clean['LOCATION_TYPE'] = df_clean['LOCATION_TYPE'].fillna('UNKNOWN')
    df_clean['PREMISES_TYPE'] = df_clean['PREMISES_TYPE'].fillna('Other')
    
    df_clean['REPORT_DELAY'] = (df_clean['REPORT_DATE'] - df_clean['OCC_DATE']).dt.total_seconds() / 3600
    
    df_clean['BIKE_MAKE'] = df_clean['BIKE_MAKE'].str.strip().str.upper()
    df_clean['BIKE_MODEL'] = df_clean['BIKE_MODEL'].str.strip().str.upper()
        
    return df_clean
    

df_cleaned = clean_bicycle_data(df)

print(df_cleaned.info())

null_counts = df_cleaned.isnull().sum()
print("\nSố lượng giá trị null trong mỗi cột:")
print(null_counts[null_counts > 0])

df_cleaned.to_csv('../src/data/clean_data.csv', index=False)


PREMISES_TYPE
Outside        10835
Apartment       8851
House           5335
Other           4473
Commercial      4279
Educational     1578
Transit          774
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36125 entries, 0 to 36124
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   OBJECTID           36125 non-null  int64         
 1   EVENT_UNIQUE_ID    36125 non-null  object        
 2   PRIMARY_OFFENCE    36125 non-null  object        
 3   OCC_DATE           36125 non-null  datetime64[ns]
 4   OCC_YEAR           36125 non-null  int64         
 5   OCC_MONTH          36125 non-null  object        
 6   OCC_DOW            36125 non-null  object        
 7   OCC_DAY            36125 non-null  int64         
 8   OCC_DOY            36125 non-null  int64         
 9   OCC_HOUR           36125 non-null  int64         
 10  REPORT_DATE        36125 non-null  date

In [53]:
import pandas as pd
df = pd.read_csv('../src/data/clean_data.csv')
df['BIKE_COST'].value_counts()


BIKE_COST
0.0       4165
500.0     2646
1000.0    2180
600.0     1728
800.0     1471
          ... 
1089.0       1
858.8        1
461.0        1
945.0        1
634.0        1
Name: count, Length: 2146, dtype: int64

In [ ]:
import pandas as pd

data =pd.read_csv('../data_ingestion_service/src/data/Bicycle_Thefts_Open_Data.csv')

In [6]:
import pandas as pd
from datetime import datetime

def split_csv_by_date(input_file, cutoff_date):
    # Read the CSV file
    df = pd.read_csv(input_file)
    
    # Convert OCC_DATE to datetime
    df['OCC_DATE'] = pd.to_datetime(df['OCC_DATE'])
    
    # Convert cutoff_date string to datetime
    cutoff_date = pd.to_datetime(cutoff_date)
    
    # Split the data
    before_cutoff = df[df['OCC_DATE'] < cutoff_date]
    after_cutoff = df[df['OCC_DATE'] >= cutoff_date]
    
    # Generate output filenames
    base_name = input_file.rsplit('.', 1)[0]
    before_file = f"{base_name}_before_sept2024.csv"
    after_file = f"{base_name}_after_sept2024.csv"
    
    # Save to separate CSV files
    before_cutoff.to_csv(before_file, index=False)
    after_cutoff.to_csv(after_file, index=False)
    
    # Print summary
    print(f"Total records: {len(df)}")
    print(f"Records before {cutoff_date.date()}: {len(before_cutoff)}")
    print(f"Records after {cutoff_date.date()}: {len(after_cutoff)}")
    print(f"\nFiles created:")
    print(f"- {before_file}")
    print(f"- {after_file}")

# Define the cutoff date
cutoff_date = '2024-09-01'

# Assuming your input file is named 'input.csv'
input_file = '../data_ingestion_service/src/data/Bicycle_Thefts_Open_Data.csv'

# Run the function
split_csv_by_date(input_file, cutoff_date)

Total records: 37178
Records before 2024-09-01: 36907
Records after 2024-09-01: 271

Files created:
- ../data_ingestion_service/src/data/Bicycle_Thefts_Open_Data_before_sept2024.csv
- ../data_ingestion_service/src/data/Bicycle_Thefts_Open_Data_after_sept2024.csv
